# StyleTTS Demo (LJSpeech)


### Utils

In [3]:
import sys
import os

# Add current directory explicitly
sys.path.insert(0, os.getcwd())

# Print to verify Python sees the correct directory
print("Current Working Directory:", os.getcwd())
print("Python Path:", sys.path)

# Now import
from models import *  
from utils import *


Current Working Directory: c:\Users\garym\OneDrive\Scripts\GM_Alienware\workspaces\StyleTTS
Python Path: ['c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\workspaces\\StyleTTS', 'c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\opt\\Conda\\envs\\styletts\\python39.zip', 'c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\opt\\Conda\\envs\\styletts\\DLLs', 'c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\opt\\Conda\\envs\\styletts\\lib', 'c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\opt\\Conda\\envs\\styletts', '', 'c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\opt\\Conda\\envs\\styletts\\lib\\site-packages', 'c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\opt\\Conda\\envs\\styletts\\lib\\site-packages\\win32', 'c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\opt\\Conda\\envs\\styletts\\lib\\site-packages\\win32\\lib', 'c:\\Users\\garym\\OneDrive\\Scripts\\GM_Alienware\\opt\\Conda\\envs\\styletts\\lib\\site-packages\\Pythonwin']


In [4]:
# Imports and setup for garo_Inference_LJSpeech.ipynb
import os
import sys

sys.path.insert(0, os.getcwd())  # Ensure the working directory is first in sys.path

import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize
from models import *  # This should now work correctly
from utils import *
# %matplotlib inline # Keep or remove as needed
print("✅ Imports successful! All modules are loaded.")

✅ Imports successful! All modules are loaded.


In [5]:
# This is mostly for verification, not setup.  The environment
# variables should be set *before* launching the notebook.
from phonemizer.backend import EspeakBackend

try:
    backend = EspeakBackend("en-us")
    print("✅ eSpeak is properly detected by Phonemizer!")
except Exception as e:
    print("❌ eSpeak detection failed!", e)

✅ eSpeak is properly detected by Phonemizer!


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [7]:
_pad = "$"
_punctuation = ';:,.!?¡¿—…"«»“” '
_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
_letters_ipa = "ɑɐɒæɓʙβɔɕçɗɖðʤəɘɚɛɜɝɞɟʄɡɠɢʛɦɧħɥʜɨɪʝɭɬɫɮʟɱɯɰŋɳɲɴøɵɸθœɶʘɹɺɾɻʀʁɽʂʃʈʧʉʊʋⱱʌɣɤʍχʎʏʑʐʒʔʡʕʢǀǁǂǃˈˌːˑʼʴʰʱʲʷˠˤ˞↓↑→↗↘'̩'ᵻ"

# Export all symbols:
symbols = [_pad] + list(_punctuation) + list(_letters) + list(_letters_ipa)

dicts = {symbols[i]: i for i in range(len(symbols))}

class TextCleaner:
    def __init__(self, dummy=None):
        self.word_index_dictionary = dicts

    def __call__(self, text):
        indexes = []
        for char in text:
            try:
                indexes.append(self.word_index_dictionary[char])
            except KeyError:
                print(f"⚠️ Warning: Character '{char}' not found in dictionary!")
        return indexes

# Initialize the text cleaner
textclenaer = TextCleaner()

# ✅ Test it immediately
test_text = "Hello, world!"
cleaned_text = textclenaer(test_text)

print("✅ Text Cleaner is working! Processed Output:", cleaned_text)


✅ Text Cleaner is working! Processed Output: [24, 47, 54, 54, 57, 3, 16, 65, 57, 60, 54, 46, 5]


In [8]:
# Define Mel Spectrogram transformation
to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300
)
mean, std = -4, 4

# Function to create a mask for padding
def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask + 1, lengths.unsqueeze(1))
    return mask

# Preprocess waveform into Mel spectrogram
def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

# Compute style embeddings for reference audio files
def compute_style(ref_dicts, model):
    reference_embeddings = {}
    for key, path in ref_dicts.items():
        wave, sr = librosa.load(path, sr=24000)
        audio, index = librosa.effects.trim(wave, top_db=30)
        if sr != 24000:
            audio = librosa.resample(audio, sr, 24000)
        mel_tensor = preprocess(audio).to(device)

        with torch.no_grad():
            ref = model.style_encoder(mel_tensor.unsqueeze(1))
        reference_embeddings[key] = (ref.squeeze(1), audio)
    
    return reference_embeddings

# ✅ Test the preprocess function with random noise
test_wave = np.random.randn(24000)  # 1 second of fake audio
mel_output = preprocess(test_wave)

print("✅ Preprocessing test successful! Mel Shape:", mel_output.shape)


✅ Preprocessing test successful! Mel Shape: torch.Size([1, 80, 81])


### Load models

In [9]:
# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)
test_text = "Hello, world!"
phonemes = global_phonemizer.phonemize([test_text])

print("✅ Phonemizer test successful! Output:", phonemes)


✅ Phonemizer test successful! Output: ['həlˈoʊ, wˈɜːld! ']


In [10]:
# load hifi-gan
import sys
sys.path.insert(0, "../Demo/hifi-gan")
sys.path.append(r"C:\Users\garym\OneDrive\Scripts\GM_Alienware\workspaces\StyleTTS\Demo\hifi-gan")

import glob
import os
import argparse
import json
import torch
from scipy.io.wavfile import write
from attrdict import AttrDict
from vocoder import Generator
print("✅ Successfully imported vocoder!")
import librosa
import numpy as np
import torchaudio

h = None

def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device, weights_only=True)
    print("Complete.")
    return checkpoint_dict

def scan_checkpoint(cp_dir, prefix):
    pattern = os.path.join(cp_dir, prefix + '*')
    cp_list = glob.glob(pattern)
    if len(cp_list) == 0:
        return ''
    return sorted(cp_list)[-1]

cp_g = scan_checkpoint("Vocoder/", 'g_')

config_file = os.path.join(os.path.split(cp_g)[0], 'config.json')
with open(config_file) as f:
    data = f.read()
json_config = json.loads(data)
h = AttrDict(json_config)

device = torch.device(device)
generator = Generator(h).to(device)

state_dict_g = load_checkpoint(cp_g, device)
generator.load_state_dict(state_dict_g['generator'])
generator.eval()
generator.remove_weight_norm()

✅ Successfully imported vocoder!


c:\Users\garym\OneDrive\Scripts\GM_Alienware\opt\Conda\envs\styletts\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loading 'Vocoder\g_00750000'
Complete.
Removing weight norm...


In [31]:
from munch import munchify  # Recursively converts dicts to Munch objects

def load_partial_state_dict(module, state_dict):
    """
    Loads parameters from state_dict into module for those keys that match in shape.
    """
    model_dict = module.state_dict()
    filtered_dict = {}
    for name, param in state_dict.items():
        if name in model_dict:
            if param.shape == model_dict[name].shape:
                filtered_dict[name] = param
            else:
                print(f"Skipping parameter '{name}' due to shape mismatch: "
                      f"checkpoint shape {param.shape}, model shape {model_dict[name].shape}")
        else:
            print(f"Parameter '{name}' not found in model; skipping.")
    # Update the module's state dict with the filtered values and load with strict=False
    model_dict.update(filtered_dict)
    module.load_state_dict(model_dict, strict=False)
    return

# Load StyleTTS
model_path = "./Models/LJSpeech/epoch_2nd_00180.pth"
model_config_path = "./Models/LJSpeech/config.yml"

# Recursively convert the configuration dict to a Munch object
config = munchify(yaml.safe_load(open(model_config_path)))

# Load pretrained ASR model
text_aligner = load_ASR_models(config.ASR_path, config.ASR_config)

# Load pretrained F0 model
pitch_extractor = load_F0_models(config.F0_path)

# ✅ Fix: Load a default BERT model since it's missing in config.yml
from transformers import AutoModel, AutoConfig

bert_model_name = "bert-base-uncased"  # Default BERT model
bert_config = AutoConfig.from_pretrained(bert_model_name)
bert = AutoModel.from_pretrained(bert_model_name, config=bert_config)
print(config.model_params.keys())  # Check which keys exist

# Ensure the necessary keys exist in model_params (they are now Munch objects)
if not hasattr(config.model_params, 'decoder'):
    print("⚠️ Warning: 'decoder' is missing in config! Using default settings.")
    config.model_params.decoder = munchify({
        'type': 'hifigan',
        'resblock_kernel_sizes': [3, 7, 11],
        'upsample_rates': [10, 5, 3, 2],
        'upsample_initial_channel': 512,
        'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
        'upsample_kernel_sizes': [20, 10, 6, 4]
    })

if not hasattr(config.model_params, 'max_dur'):
    print("⚠️ Warning: 'max_dur' is missing in config! Using default value: 1 (to match pretrained model).")
    config.model_params.max_dur = 1

if not hasattr(config.model_params, 'multispeaker'):
    print("⚠️ Warning: 'multispeaker' is missing in config! Using default value: False.")
    config.model_params.multispeaker = False

if not hasattr(config.model_params, 'diffusion'):
    print("⚠️ Warning: 'diffusion' is missing in config! Using default with transformer and embedding_mask_proba.")
    config.model_params.diffusion = munchify({
        'transformer': {
            'num_layers': 2,
            'num_heads': 8,
            'head_features': 64,
            'multiplier': 4
        },
        'dist': {
            'mean': 0.0,
            'std': 1.0,
            'sigma_data': 0.5
        },
        'embedding_mask_proba': 0.1
    })

if not hasattr(config.model_params, 'slm'):
    print("⚠️ Warning: 'slm' is missing in config! Using default values.")
    config.model_params.slm = munchify({
        'hidden': 256,
        'nlayers': 2,
        'initial_channel': 512
    })

# Now call build_model() with the proper configuration
model = build_model(config.model_params, text_aligner, pitch_extractor, bert)

# Load model parameters from checkpoint
params = torch.load(model_path, map_location='cpu')['net']
for key in model:
    if key in params:
        if "discriminator" not in key:
            print(f"{key} loaded")
            load_partial_state_dict(model[key], params[key])

# Set model to evaluation mode and move to device
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]


dict_keys(['hidden_dim', 'n_token', 'style_dim', 'n_layer', 'dim_in', 'max_conv_dim', 'n_mels', 'dropout', 'decoder'])
⚠️ Warning: 'max_dur' is missing in config! Using default value: 1 (to match pretrained model).
⚠️ Warning: 'multispeaker' is missing in config! Using default value: False.
⚠️ Warning: 'diffusion' is missing in config! Using default with transformer and embedding_mask_proba.
⚠️ Warning: 'slm' is missing in config! Using default values.


C:\Users\garym\AppData\Local\Temp\ipykernel_17516\3861486645.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(model_path, map_location='cpu')['net']


predictor loaded
decoder loaded
Skipping parameter 'decode.2.conv1.bias' due to shape mismatch: checkpoint shape torch.Size([512]), model shape torch.Size([1024])
Skipping parameter 'decode.2.conv1.weight_g' due to shape mismatch: checkpoint shape torch.Size([512, 1, 1]), model shape torch.Size([1024, 1, 1])
Skipping parameter 'decode.2.conv1.weight_v' due to shape mismatch: checkpoint shape torch.Size([512, 1090, 3]), model shape torch.Size([1024, 1090, 3])
Skipping parameter 'decode.2.conv2.bias' due to shape mismatch: checkpoint shape torch.Size([512]), model shape torch.Size([1024])
Skipping parameter 'decode.2.conv2.weight_g' due to shape mismatch: checkpoint shape torch.Size([512, 1, 1]), model shape torch.Size([1024, 1, 1])
Skipping parameter 'decode.2.conv2.weight_v' due to shape mismatch: checkpoint shape torch.Size([512, 512, 3]), model shape torch.Size([1024, 1024, 3])
Skipping parameter 'decode.2.norm2.fc.weight' due to shape mismatch: checkpoint shape torch.Size([1024, 128

### Synthesize speech

In [ ]:
# get first 3 training sample as references

train_path = config.get('train_data', None)
val_path = config.get('val_data', None)
train_list, val_list = get_data_path_list(train_path, val_path)

ref_dicts = {}
for j in range(3):
    filename = train_list[j].split('|')[0]
    name = filename.split('/')[-1].replace('.wav', '')
    ref_dicts[name] = filename
    
reference_embeddings = compute_style(ref_dicts, model)

In [ ]:
# synthesize a text
text = ''' StyleTTS is a style-based generative model for parallel TTS that can synthesize diverse speech with natural prosody from a reference speech utterance. '''

In [ ]:
# tokenize
ps = global_phonemizer.phonemize([text])
ps = word_tokenize(ps[0])
ps = ' '.join(ps)
tokens = textclenaer(ps)
tokens.insert(0, 0)
tokens.append(0)
tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

In [ ]:
converted_samples = {}

with torch.no_grad():
    input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
    m = length_to_mask(input_lengths).to(device)
    t_en = model.text_encoder(tokens, input_lengths, m)
        
    for key, (ref, _) in reference_embeddings.items():
        
        s = ref.squeeze(1)
        style = s
        
        d = model.predictor.text_encoder(t_en, style, input_lengths, m)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)
        
        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        style = s.expand(en.shape[0], en.shape[1], -1)

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        out = model.decoder((t_en @ pred_aln_trg.unsqueeze(0).to(device)), 
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))


        c = out.squeeze()
        y_g_hat = generator(c.unsqueeze(0))
        y_out = y_g_hat.squeeze().cpu().numpy()

        c = out.squeeze()
        y_g_hat = generator(c.unsqueeze(0))
        y_out = y_g_hat.squeeze()
        
        converted_samples[key] = y_out.cpu().numpy()

In [ ]:
import IPython.display as ipd
for key, wave in converted_samples.items():
    print('Synthesized: %s' % key)
    display(ipd.Audio(wave, rate=24000))
    try:
        print('Reference: %s' % key)
        display(ipd.Audio(reference_embeddings[key][-1], rate=24000))
    except:
        continue